In [8]:
# import libraries and utilities
from object_detection_models import *
from object_detection_utils import *
from video_utils import *
from math_utils import MathUtils
from lane_detection import LaneDetection
from road_entity_detection import RoadEntityDetection
import os, sys, time, datetime, random
import cv2
import torch 
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

In [9]:
# CarCam computer vision pipeline
def carcam_pipeline(inputvid_filename, outputvid_filename):
    print('Input video: {}'.format(inputvid_filename))
    print('Executing CarCam computer vision pipeline. Please wait...')
    frames = extract_frames(inputvid_filename)
    processed_frames, lane_lines = LaneDetection.detect_lanes(frames)
    processed_frames = RoadEntityDetection.detect_road_entities(processed_frames, lane_lines)
    compile_frames(processed_frames, outputvid_filename)
    cv2.destroyAllWindows()
    print('Output video: {}'.format(outputvid_filename))
    print('All done, check out your output video!')

In [10]:
# execute pipeline
carcam_pipeline('input.mov', 'output.mp4')

Input video: input.mov
Executing CarCam computer vision pipeline. Please wait...


NameError: name 'extract_frames' is not defined